In [4]:
import numpy as np

In [ ]:
class Zonotope:
    def __init__(self, centre, generators):
        self.centre = np.array(centre).reshape(-1)         # shape (n,)
        self.generators = np.atleast_2d(generators)        # shape (n, m)
        if self.generators.shape[0] != self.centre.shape[0]:
            raise ValueError("Generators must have same dimension as centre")
        self.d = self.centre.shape[0]   # dimension
        self.m = self.generators.shape[1]  # number of generators

    def minkowski_sum(self, other):
        new_centre = self.centre + other.centre
        new_generators = np.hstack((self.generators, other.generators))
        return Zonotope(new_centre, new_generators)
    
    def subtract(self, other):
        new_centre = self.centre - other.centre
        new_generators = np.hstack((self.generators, -other.generators))
        return Zonotope(new_centre, new_generators)
    
    @classmethod
    def from_intervals(cls, lower, upper):
        lower = np.array(lower).reshape(-1)
        upper = np.array(upper).reshape(-1)

        if lower.shape != upper.shape:
            raise ValueError("Lower and upper must have same shape")

        centre = (lower + upper) / 2
        radii = (upper - lower) / 2

        generators = np.diag(radii)
        return cls(centre, generators)
    
    def output_interval(self):
        radius = np.sum(np.abs(self.generators), axis=1)  # sum across generators
        lower = self.centre - radius
        upper = self.centre + radius
        return lower, upper

    def affine_map(self, A, b=None):
            # Ensure centre is a column vector for matrix multiplication
            new_centre = A @ self.centre
            if b is not None:
                new_centre = new_centre + np.array(b).reshape(-1)
            new_generators = A @ self.generators
            return Zonotope(new_centre, new_generators)
    
    def __repr__(self):
        return f"Zonotope(dim={self.d}, generators={self.m})\nCentre:\n{self.centre}\nGenerators:\n{self.generators}"

Zonotope(dim=4, generators=3)
Centre:
[-0.33997547  0.00631931 -0.17244171  1.58108294]
Generators:
[[-0.02429142  0.69480798  0.47504854]
 [-0.03810278  0.30454001  0.64378823]
 [-0.25023542 -0.52010569 -0.23582511]
 [ 0.07577974 -0.71538686  1.93511821]]
